# Measuring the correlation between data content and filenames

#### Author: **Thomas Casey**
##### Location: University of California Santa Barbara

### **Table of Contents**
1. [Statement of Purpose](#Statement-of-Purpose)
2. [Nature of the Data](#Nature-of-the-Data)
3. [Available Tools](#Available-Tools)
4. [Method Description](#Method-Description)
5. [Application](#Application)
7. [Conclusions](#Conclusions)

<a id="Statement of Purpose"> </a>
### Statement of Purpose

Can undocumented and unstructured data be structured and interpreted with an acceptable level of confidence based only on the raw content of the data and filenames associated with the data?


<a id="Nature of the Data"> </a>
### Nature of the Data

insert

<a id="Available Tools"> </a>
### Available Tools

insert

<a id="Method Description"> </a>
### Method Description

**Retrieve and sort data**

In [1]:
# some code

**Pre-process**

In [ ]:
# some code

**Define model**

In [ ]:
# some code

**Application**

In [ ]:
# some code

**Results Interpretation**

In [ ]:
# some code

<a id="Application"> </a>
### Application

insert

In [2]:
# some code

<a id="Conclusions"> </a>
### Conclusions

insert